<a href="https://colab.research.google.com/github/raymondwcs/learning_bert/blob/master/Sentiment_Analysis_(hugging_face_pipeline%2C_uer_roberta_base_finetuned_jd_full_chinese).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet transformers
from transformers import AutoModelForSequenceClassification,AutoTokenizer,pipeline

     |████████████████████████████████| 2.6 MB 4.4 MB/s 
     |████████████████████████████████| 636 kB 58.9 MB/s 
     |████████████████████████████████| 895 kB 53.1 MB/s 
     |████████████████████████████████| 3.3 MB 40.3 MB/s 


In [62]:
# checkpoint = 'uer/roberta-base-finetuned-jd-full-chinese'  # https://huggingface.co/uer/roberta-base-finetuned-jd-full-chinese
checkpint = 'bert-base-chinese'
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)
sentences = [
  '芒果大合奏~ 次次去都係食呢個，好多層次的芒果，有芒果乾，芒果布丁，芒果肉，芒果雪糕，芒果漿，不同texture, 感覺真係好rich~ 大滿足',
  '但係我地黎既目的係食韓燒，點知種類賣相真係嚇死你！啲肉好似冇奄過咁！啲牛肉粒仲瘦過瘦肉，都算啦！點知所有野食燒完後完全冇味！',
  '這個服務生很不親切'
]

classifier(sentences)


[{'label': 'star 5', 'score': 0.8894888162612915},
 {'label': 'star 2', 'score': 0.5215009450912476},
 {'label': 'star 1', 'score': 0.39362475275993347}]

# Sentence Embedding in BERT

In [57]:
sentences = [
  '這個服務生很不親切',
  '这个服务生很不亲切', 
  '今天是晴天今天是晴',
  '今天是晴天',
  '今天陽光充沛!',
  '今天下大雨',     
]

from transformers import BertModel

model = BertModel.from_pretrained(checkpoint)

tokens = tokenizer(text=sentences, add_special_tokens=True, padding=True, truncation=True, return_tensors='pt')

input_ids = tokens.input_ids[0].unsqueeze(0)
attention_mask = tokens.attention_mask[0].unsqueeze(0)
output = model(input_ids=input_ids, attention_mask=attention_mask)
sentence_embedding_01 = output.pooler_output

input_ids = tokens.input_ids[1].unsqueeze(0)
attention_mask = tokens.attention_mask[1].unsqueeze(0)
output = model(input_ids=input_ids, attention_mask=attention_mask)
sentence_embedding_02 = output.pooler_output

import torch
from torch.nn.functional import cosine_similarity, pairwise_distance

print(cosine_similarity(sentence_embedding_01, sentence_embedding_02))
print(pairwise_distance(sentence_embedding_01, sentence_embedding_02))

Some weights of the model checkpoint at uer/roberta-base-finetuned-jd-full-chinese were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([0.9979], grad_fn=<DivBackward0>)
tensor([0.8085], grad_fn=<NormBackward1>)


In [4]:
output = model(input_ids=tokens.input_ids, output_hidden_states=True)

# https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/
# output[0]   # last_hidden_state
# output[1]   # pooler_output
# output[2]   # hidden layers (13 = input embeddings + 12 BERT layers)

hidden_states = output[2]

# print(hidden_states)
print(len(hidden_states))             # 13 = input embeddings + 12 BERT layers
print(hidden_states[-1].shape)        # last hidden layer [1, <input length>, 768]
print(len(hidden_states[-1]))         # 1
print(len(hidden_states[-1][0]))      # <input length>
print(len(hidden_states[-1][0][0]))   # 768
print(hidden_states[-1])

# `hidden_states` has shape [13 x 1 x <input length> x 768]
# `token_vecs` is a tensor with shape [<input length> x 768]
token_vecs = hidden_states[1:13][0]
print(len(token_vecs))

# Calculate the average of all token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)

print ("Our final sentence embedding vector of shape:", sentence_embedding.size())


13
torch.Size([6, 11, 768])
6
11
768
tensor([[[ 0.0627, -0.0396, -0.0781,  ...,  0.0085,  0.2588, -0.6025],
         [ 0.2348,  0.0642, -0.0767,  ...,  0.1848,  0.4004, -0.3693],
         [ 0.2111,  0.0940, -0.0319,  ...,  0.1745,  0.3543, -0.2807],
         ...,
         [ 0.1526, -0.0884, -0.0880,  ...,  0.0538,  0.2824, -0.3470],
         [ 0.1402, -0.0732, -0.0842,  ...,  0.0567,  0.2550, -0.3416],
         [ 0.0627, -0.0396, -0.0781,  ...,  0.0085,  0.2588, -0.6025]],

        [[ 0.0344, -0.0613, -0.0784,  ..., -0.0095,  0.2935, -0.5952],
         [ 0.1838,  0.0070, -0.0668,  ...,  0.1731,  0.4417, -0.3823],
         [ 0.1603,  0.0282, -0.0163,  ...,  0.1786,  0.3784, -0.2861],
         ...,
         [ 0.1448, -0.1205, -0.0857,  ...,  0.0273,  0.2931, -0.3139],
         [ 0.1378, -0.0929, -0.0832,  ...,  0.0504,  0.2634, -0.3347],
         [ 0.0344, -0.0613, -0.0784,  ..., -0.0096,  0.2935, -0.5952]],

        [[ 0.0187,  0.0908,  0.0059,  ..., -0.1731,  0.1371,  0.1081],
        

In [58]:
tokens = tokenizer(text=sentences, add_special_tokens=True, padding=True, truncation=True, return_tensors='pt')


input_ids = tokens.input_ids[0].unsqueeze(0)
attention_mask = tokens.attention_mask[2].unsqueeze(0)
output = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
hidden_states = output[2]
token_vecs = hidden_states[1:][0]
sentence_embedding01= torch.mean(token_vecs,dim=0)

input_ids = tokens.input_ids[1].unsqueeze(0)
attention_mask = tokens.attention_mask[1].unsqueeze(0)
output = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
hidden_states = output[2]
token_vecs = hidden_states[1:][0]
sentence_embedding02= torch.mean(token_vecs,dim=0)

print(sentence_embedding01.size())
print(sentence_embedding02.size())

import torch
from torch.nn.functional import cosine_similarity, pairwise_distance

sim = cosine_similarity(sentence_embedding01, sentence_embedding02)
torch.mean(sim, dim=0)

torch.Size([11, 768])
torch.Size([11, 768])


tensor(0.9280, grad_fn=<MeanBackward1>)